# Time switch: include last 2 years in training, first 2 years as test

Rationale: the last 2 years of testing data contain highs and lows in C* not earlier seen in the trainig data. Therefore, it is hypothesised that if these were to be included in the training, better test performances might be reached.

In [ ]:
import os
import pandas as pd
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == "ml_observation_operator":
    pad_correct = pad.parent
    os.chdir(pad_correct)
%run "ml_observation_operator/data_load_in.py"

In [ ]:
ML_data_pad = Path("data/Zwalm_data/ML_data")
X_full_all = pd.read_pickle(ML_data_pad/"X_full_all.pkl")

y_train = pd.read_pickle(ML_data_pad/"y_train_bis.pkl")
y_test = pd.read_pickle(ML_data_pad/"y_test_bis.pkl")
y_full = pd.read_pickle(ML_data_pad/"y_full.pkl")

X_train = pd.read_pickle(ML_data_pad/"X_train_bis.pkl")
X_test = pd.read_pickle(ML_data_pad/"X_test_bis.pkl")
X_full = pd.read_pickle(ML_data_pad/"X_full.pkl")

X_train_small = pd.read_pickle(ML_data_pad/"X_train_small_bis.pkl")
X_test_small = pd.read_pickle(ML_data_pad/"X_test_small_bis.pkl")
X_full_small = pd.read_pickle(ML_data_pad/"X_full_small.pkl")

Cstar = pd.read_pickle(ML_data_pad/"Cstar.pkl")

Drop delta t 

In [ ]:
X_train = X_train.drop('delta_t',axis = 1)
X_test = X_test.drop('delta_t',axis = 1)
X_full = X_full.drop('delta_t',axis = 1)
display(X_full.head())

X_train_small = X_train_small.drop('delta_t',axis = 1)
X_test_small = X_test_small.drop('delta_t',axis = 1)
X_full_small = X_full_small.drop('delta_t',axis = 1)
display(X_full_small.head())

## Load in used packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as python_random
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV,   LassoCV 
from sklearn.model_selection import KFold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from functions.ml_utils import general_sklearn_model, validation_loop, full_training_loop
from functions.pre_processing import reshape_data, reshaped_to_train_test
from functions.plotting_functions import ensemble_plot
SEED = 1234

python_random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
%load_ext autoreload
%autoreload 2

In [ ]:
os.getcwd()

In [ ]:
import rasterio

## Load in data

In [ ]:
%run "ml_observation_operator/data_load_in.py"

In [ ]:
ML_data_pad = Path("data/Zwalm_data/ML_data")
X_full_all = pd.read_pickle(ML_data_pad/"X_full_all.pkl")

y_train = pd.read_pickle(ML_data_pad/"y_train_bis.pkl")
y_test = pd.read_pickle(ML_data_pad/"y_test_bis.pkl")
y_full = pd.read_pickle(ML_data_pad/"y_full.pkl")

X_train = pd.read_pickle(ML_data_pad/"X_train_bis.pkl")
X_test = pd.read_pickle(ML_data_pad/"X_test_bis.pkl")
X_full = pd.read_pickle(ML_data_pad/"X_full.pkl")

X_train_small = pd.read_pickle(ML_data_pad/"X_train_small_bis.pkl")
X_test_small = pd.read_pickle(ML_data_pad/"X_test_small_bis.pkl")
X_full_small = pd.read_pickle(ML_data_pad/"X_full_small.pkl")

Cstar = pd.read_pickle(ML_data_pad/"Cstar.pkl")

Drop the delta t feature again: only relevant for LSTM

In [ ]:
X_train = X_train.drop('delta_t',axis = 1)
X_test = X_test.drop('delta_t',axis = 1)
X_full = X_full.drop('delta_t',axis = 1)
display(X_full.head())

X_train_small = X_train_small.drop('delta_t',axis = 1)
X_test_small = X_test_small.drop('delta_t',axis = 1)
X_full_small = X_full_small.drop('delta_t',axis = 1)
display(X_full_small.head())

In [ ]:
# A normalised version for when needed
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_full_norm = X_scaler.transform(X_full)

y_scaler = StandardScaler()
y_scaler.fit(y_train.values.reshape(-1,1))
y_full_norm = y_scaler.transform(y_full.values.reshape(-1,1))

# Linear regression

In [ ]:
linreg, r2_train, r2_test, fig, ax = general_sklearn_model(LinearRegression(),
    X_train,X_test, y_train.values.reshape(-1,1), y_test.values.reshape(-1,1), X_train.index,X_test.index, Cstar,normalisation = True)

In [ ]:
linreg.coef_

Drop time feature

In [ ]:
linreg_drop, r2_train, r2_test, fig, ax = general_sklearn_model(
     LinearRegression(), X_train.drop(['year_sin','year_cos'], axis = 1),
     X_test.drop(['year_sin','year_cos'], axis = 1), 
     y_train.values.reshape(-1,1),y_test.values.reshape(-1,1),X_train.index, X_test.index, Cstar)

# Ridge regression on window

Best model structure for ridge regresion on window (cf `simple_models.ipynb'). But use a different alpha!

In [ ]:
pad = Path('data/ml_obs_op_data/ridge/window')
hyperparam_best_nt = pd.read_csv(pad/'best_hyperparam.csv')
model_name = hyperparam_best_nt['model'].values[0]
alpha = hyperparam_best_nt['alpha'].values[0]
seq_length = hyperparam_best_nt['seq_length'].values[0]
time_goniometr = hyperparam_best_nt['time_bool'].values[0]
forest = hyperparam_best_nt['forest_bool'].values[0]
X_temp_train = X_train.copy()
X_temp_test = X_test.copy()
if not time_goniometr:
    X_temp_train = X_temp_train.drop(['year_sin','year_cos'],axis = 1)
    X_temp_test = X_temp_test.drop(['year_sin','year_cos'],axis = 1)
if not forest:
    X_temp_train = X_temp_train.loc[:,~X_temp_train.columns.str.endswith('Forest')]
    X_temp_test = X_temp_test.loc[:,~X_temp_test.columns.str.endswith('Forest')]
#kf = KFold(n_splits = 5,shuffle= True, random_state = SEED)
if model_name == 'Ridge':
    model = RidgeCV(alphas = np.logspace(-3,3,100), cv = 5)
elif model_name == 'Lasso':
    model = LassoCV(alphas = np.logspace(-3,3,100), cv = 5)
else:
    ValueError("Unknown model name")


In [ ]:
#Custom window creating! Neede for correct use of 
# X_W_full, y_w_full, t_w_full= reshape_data(X_full.values, y_full.values.reshape(-1,1), X_full.index,seq_length)
X_W_full, y_w_full, t_w_full= reshape_data(X_full_norm, y_full_norm, X_full.index,seq_length)
#Just use reshaped_to_train_test where you train -> test and test -> train with n_train = n_test!
n_test_og = len(y_test)
(X_test_w, X_train_w, y_test_w, y_train_w, t_test_w, 
 t_train_w) = reshaped_to_train_test(X_W_full, y_w_full, t_w_full, seq_length, n_test_og,output_dim=2)

In [ ]:
X_train_w.shape

In [ ]:

func, r2_train, r2_test, fig, ax, y_train_hat, y_test_hat = general_sklearn_model(model, X_train_w, X_test_w, y_train_w, y_test_w,
    t_train_w, t_test_w, Cstar,normalisation = False,seq_length = seq_length,
    print_output = False, return_predictions=True)
fig, ax = plt.subplots()
Cstar[X_full.index].plot(ax=ax)
ax.plot(y_train_hat.index, y_scaler.inverse_transform(y_train_hat.values), label='Train')
ax.plot(y_test_hat.index, y_scaler.inverse_transform(y_test_hat.values), label='Test')
ax.legend()
ax.set_ylabel('C* [mm]')
print(r2_train)
print(r2_test)

In [ ]:
func, r2_train, r2_test, fig, ax, y_train_hat, y_test_hat = general_sklearn_model(Ridge(alpha = alpha), X_train_w, X_test_w, y_train_w, y_test_w,t_train_w, t_test_w, Cstar,normalisation = False,seq_length = seq_length, return_predictions=True, print_output=False)
fig, ax = plt.subplots()
Cstar[X_full.index].plot(ax=ax)
ax.plot(y_train_hat.index, y_scaler.inverse_transform(y_train_hat.values), label='Train')
ax.plot(y_test_hat.index, y_scaler.inverse_transform(y_test_hat.values), label='Test')
ax.legend()
ax.set_ylabel('C* [mm]')
print(r2_train)
print(r2_test)

In [ ]:
t_test_w.shape

An overfit function now

# Non-linear: GP RBF

In [ ]:
kernel = RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(
    noise_level=1, noise_level_bounds=(1e-2, 1e3) #1e-1 seems best as lower
)
gpr = GaussianProcessRegressor(kernel = kernel, alpha= 0, n_restarts_optimizer=100, random_state=SEED)
gpr_for_pipe = GaussianProcessRegressor(kernel = kernel, alpha= 0, n_restarts_optimizer=100, normalize_y = True, random_state=SEED)
gpr_pipe = make_pipeline(StandardScaler(),gpr_for_pipe) 
# normalisation can be included in general GP (easier than via normalisation in general sklearn model)
#Pad to save to later
pad = Path('data/ml_obs_op_data/GPR_bis')
gpr_out, r2_train, r2_test, fig, ax = general_sklearn_model(
     gpr_pipe, X_train, X_test, y_train, y_test, X_train.index, X_test.index, Cstar,normalisation=False, save_predictions=True, pad = pad
)
ax.set_title('GP RBF full datastet')

Worse performer than Linear Regression

In [ ]:
gpr_out[1].kernel_

# Test the ANN structure

In [ ]:
n_features = X_train.shape[1]
print(2*n_features+1) #round of to 32 for first layer
ann_extended = models.Sequential(
    [
    layers.Input(shape = (n_features,)),
    layers.Dense(32, activation = "relu", name = 'layer1'), #first number = dimensionality of the output space!
    layers.Dropout(0.2),
    layers.Dense(16, activation = "relu", name = 'layer2'),
    layers.Dropout(0.2),    
    layers.Dense(1, activation = "linear", name = 'layer3'),
    ]
)
pad = Path('data/ml_obs_op_data/ann/extende_ann_bis')
if not os.path.exists(pad):
    os.makedirs(pad)
ann_extended.summary()

exec_training = True
repeats = 8

In [ ]:
mean_r2_val, recommended_nr_epochs = validation_loop(
        ann_extended, repeats, X_train, X_test,y_train, y_test, Cstar,
        exec_training,pad,epochs = 100,print_output = False,
        verbose = 0, validation_split = 0.2, dask_bool = True)
print(mean_r2_val); print(recommended_nr_epochs)

In [ ]:
mean_r2_train, mean_r2_test, out_dict, models_list = full_training_loop(
 ann_extended, repeats,X_train, X_test,y_train,y_test, Cstar,
 exec_training, recommended_nr_epochs, pad, print_output = False,
 verbose = 0, validation_split=0.0
)

In [ ]:
n_std = 1
ensemble_plot(Cstar, X_full.index, out_dict, n_std)

# Test the LSTM structure

Skipped as of now, no indication that the changing around of time gives great benefit to non-linear models!!